# load lib

In [12]:
import numpy as np
import pandas as pd

# load data

In [13]:
city_data = pd.read_csv('city_data_restr.csv')
city_data

,city,x_co,y_co
0,0,0.3642,0.7770
1,1,0.2954,0.9606
2,2,0.5951,0.4647
3,3,0.6697,0.7657
4,4,0.4353,0.1709
5,5,0.2131,0.8349
6,6,0.3479,0.6984
7,7,0.4516,0.0488


In [14]:
city_position = city_data[['x_co','y_co']].to_numpy()


In [15]:
class GA(object):
    def __init__(self, DNA_size, cross_rate, mutation_rate, pop_size, ):
        ## get popuation 
        self.DNA_size = DNA_size
        self.cross_rate = cross_rate
        self.mutate_rate = mutation_rate
        self.pop_size = pop_size

        self.pop = np.vstack([np.random.permutation(DNA_size) for _ in range(pop_size)])

    def translateDNA(self, DNA, city_position):     # get cities' coord in order
        ## get dna
        line_x = np.empty_like(DNA, dtype=np.float64)
        line_y = np.empty_like(DNA, dtype=np.float64)
        for i, d in enumerate(DNA):
            city_coord = city_position[d]
            line_x[i, :] = city_coord[:, 0]
            line_y[i, :] = city_coord[:, 1]
        return line_x, line_y

    def get_distance(self, line_x, line_y):
        
        city_start_x = 0.0986
        city_start_y = 0.5891
        city_end_x = 0.7185
        city_end_y = 0.8312
        
        
        total_distance = np.empty((line_x.shape[0],), dtype=np.float64)
        for i, (xs, ys) in enumerate(zip(line_x, line_y)):
            total_distance[i] = np.sum(np.sqrt(np.square(np.diff(xs)) + np.square(np.diff(ys))))
            total_distance[i] = total_distance[i] +np.sum(np.sqrt(np.square(xs[0]-city_start_x)+np.square(ys[0]-city_start_y)))
            total_distance[i] = total_distance[i] +np.sum(np.sqrt(np.square(xs[-1]-city_end_x)+np.square(ys[-1]-city_end_y)))
        return total_distance
    
    def get_fitness(self, dis_list):                   #rank weighting selection
        dis_list = dict(list(enumerate(dis_list)))
        dis_list = dict(sorted(dis_list.items(), key = lambda kv:(kv[1], kv[0]),reverse=True))
        p = (np.arange(len(dis_list.values()))+1)/(np.sum(np.arange(len(dis_list.values()))+1))
        dis_list.update(zip(dis_list, p))
        dis_list = dict(sorted(dis_list.items()))
        fitness = dis_list.values()
        return list(fitness)    
    
    
    def select(self, fitness):
        idx = np.random.choice(np.arange(self.pop_size), size=self.pop_size, replace=True, p=fitness)
        return self.pop[idx]

    def crossover(self, parent, pop):
        if np.random.rand() < self.cross_rate:
            i_ = np.random.randint(0, self.pop_size, size=1)                        # select another individual from pop
            cross_points = np.random.randint(0, 2, self.DNA_size).astype(bool)   # choose crossover points
            keep_city = parent[~cross_points]                                       # find the city number
            swap_city = pop[i_, np.isin(pop[i_].ravel(), keep_city, invert=True)]
            parent[:] = np.concatenate((keep_city, swap_city))
        return parent

    def mutate(self, child):
        for point in range(self.DNA_size):
            if np.random.rand() < self.mutate_rate:
                swap_point = np.random.randint(0, self.DNA_size)
                swapA, swapB = child[point], child[swap_point]
                child[point], child[swap_point] = swapB, swapA
        return child

    def next_gen(self, fitness):
        pop = self.select(fitness)
        pop_copy = pop.copy()
        for parent in pop:  # for every parent
            child = self.crossover(parent, pop_copy)
            child = self.mutate(child)
            parent[:] = child
        self.pop = pop

 # Run GA

In [17]:
N_CITIES = 8  # DNA size
CROSS_RATE = 0.2
MUTATE_RATE = 0.03
POP_SIZE = 500
N_GENERATIONS = 500


ga = GA(DNA_size=N_CITIES, cross_rate=CROSS_RATE, mutation_rate=MUTATE_RATE, pop_size=POP_SIZE)

for generation in range(N_GENERATIONS):
    lx, ly = ga.translateDNA(ga.pop, city_position)
    total_distance = ga.get_distance(lx, ly)
    #print(total_distance)
    fitness =  ga.get_fitness(total_distance)
    ga.next_gen(fitness)
    best_idx = np.argmax(fitness)
    if ((generation+1)%10 == 0) :
        print('Gen:', generation+1, '| best total_distance: %.5f' % total_distance[best_idx],)
        print( 'best DNA:', ga.pop[best_idx] )

Gen: 10 | best total_distance: 2.18737
best DNA: [5 7 4 2 6 0 1 3]
Gen: 20 | best total_distance: 2.18737
best DNA: [7 4 2 6 0 5 1 3]
Gen: 30 | best total_distance: 2.18737
best DNA: [6 4 2 3 7 0 5 1]
Gen: 40 | best total_distance: 2.18737
best DNA: [1 0 6 4 7 3 5 2]
Gen: 50 | best total_distance: 2.18737
best DNA: [5 1 0 6 4 7 2 3]
Gen: 60 | best total_distance: 2.18737
best DNA: [5 1 0 6 4 7 2 3]
Gen: 70 | best total_distance: 2.18737
best DNA: [5 1 0 6 2 7 4 3]
Gen: 80 | best total_distance: 2.18737
best DNA: [3 1 0 6 4 7 2 5]
Gen: 90 | best total_distance: 2.18737
best DNA: [6 1 0 5 4 7 2 3]
Gen: 100 | best total_distance: 2.18737
best DNA: [5 1 0 6 4 7 2 3]
Gen: 110 | best total_distance: 2.18737
best DNA: [5 1 0 6 4 7 2 3]
Gen: 120 | best total_distance: 2.18737
best DNA: [0 1 5 6 4 7 2 3]
Gen: 130 | best total_distance: 2.18737
best DNA: [5 1 0 6 4 7 2 3]
Gen: 140 | best total_distance: 2.18737
best DNA: [5 1 0 6 4 7 2 3]
Gen: 150 | best total_distance: 2.18737
best DNA: [5 1 0 

# The shortest distance: 1.94084

# ===============================

TypeError: '(slice(0, 2, None), slice(3, 4, None))' is an invalid key